In [1]:
#这个文件用于训练一个标准的双任务5分类segnet语义分割网络
img_height, img_width = (256,256)# 输入图像尺寸
out_channel = 5# 输出频道数

print('over1')

# 导入必要的库
import os
import glob
import random
import sys
import shutil
import skimage.io
import skimage.transform
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
print('over2')

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, UpSampling2D, Reshape
from tensorflow.keras.layers import MaxPooling2D, Permute
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback


# 打印环境版本信息
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Skimage      :', skimage.__version__)
print('Scikit-learn :', sklearn.__version__)
print('Keras        :', keras.__version__)
print('Tensorflow   :', tf.__version__)

over1
over2
Python       : 3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
Numpy        : 1.23.5
Skimage      : 0.20.0
Scikit-learn : 1.1.3
Keras        : 2.4.0
Tensorflow   : 2.3.0


In [2]:
# 设置随机种子保证可重复性，使生成器对准
seed = 42
random.seed = seed
np.random.seed(seed=seed)


In [18]:
'''
topDir = os.path.dirname(os.path.abspath(__file__))# 动态获取当前脚本的父目录
__file__是Python内置变量，表示当前脚本的绝对路径。但在交互式环境中，没有"当前脚本"的概念，因此该变量不存在
如果代码保存为.py文件并直接执行，__file__可以正常使用，但需添加路径修正逻辑
'''
codDir = os.getcwd()#获取代码所在文件夹
topDir = os.path.dirname(codDir)#获取代码所在文件夹的上一级文件夹
print('项目文件夹路径:',topDir)
codDir = os.path.join(topDir, "GP_code")
print('代码文件夹路径:',codDir)
imgDir = os.path.join(topDir, "GP_img")# 获取图片所在文件夹
print('图片文件夹路径:',imgDir)
weiDir = os.path.join(topDir, "GP_wei")# 获取权重所在文件夹
print('权重文件夹路径:',weiDir)
nor5Dir = os.path.join(topDir, "nor5")
aug5Dir = os.path.join(topDir, "aug5")
aug55Dir = os.path.join(topDir, "aug55")
aug2Dir = os.path.join(topDir, "aug2")
aug25Dir = os.path.join(topDir, "aug25")

weiPre = "model-weights_pre.hdf5"#预训练权重名称
weiDur = "model-weights_dur.hdf5"#训练中权重名称
weiEnd = "model-weights_end.hdf5"#训练后权重名称

#preDir = os.path.join(aug55Dir, weiPre)# 预训练权重保存目录
#workDir = os.path.join(aug55Dir, weiDur)# 权重动态保存目录
#endDIr = os.path.join(aug55Dir, weiEnd)# 训练结束权重保存目录

os.chdir(topDir)#不要重复运行该框
print('over')

项目文件夹路径: C:\
代码文件夹路径: C:\GP_code
图片文件夹路径: C:\GP_img
权重文件夹路径: C:\GP_wei
over


In [4]:
# 定义模型结构
"""
定义函数keras_model
输入图片尺寸(img_width=256, img_height=256, out_channel)
输出一个模型model
"""
def keras_model(img_width=256, img_height=256, out_channel=out_channel):
    """
    构建多任务深度学习模型
    输入尺寸: (256, 256, 3)
    输出:
    - 分割结果: (256, 256, 1) 经过softmax激活
    - 分类结果: (4,) 四类概率分布
    结构特点:
    - 编码器使用VGG风格的卷积块
    - 解码器使用转置卷积进行上采样
    - 多任务输出：分割+分类
    """
    # 输入层配置匹配
    """
    K.image_data_format()获得输入图片格式
    """
    if K.image_data_format() == 'channels_first':
        ch_axis = 1
        input_shape = (3, img_height, img_width)
    elif K.image_data_format() == 'channels_last':
        ch_axis = 3
        input_shape = (img_height, img_width, 3)
    
    inp = Input(shape=input_shape)

    # encoder
    enc = inp
    enc = Conv2D(64, (3, 3), strides=(1, 1), input_shape=input_shape, padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = MaxPooling2D(pool_size=(2, 2))(enc)# 长宽减半
    # (128,128)
    enc = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = MaxPooling2D(pool_size=(2, 2))(enc)# 长宽减半
    # (64,64)
    enc = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = MaxPooling2D(pool_size=(2, 2))(enc)# 长宽减半
    # (32,32)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = MaxPooling2D(pool_size=(2, 2))(enc)# 长宽减半
    # (16,16)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(enc)
    enc = BatchNormalization()(enc)
    enc = MaxPooling2D(pool_size=(2, 2))(enc)# 长宽减半
    # (8,8)
    # decoder
    dec0 = UpSampling2D(size=(2, 2))(enc)# 长宽加倍
    # (16,16)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec0)
    dec = BatchNormalization()(dec)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = UpSampling2D(size=(2, 2))(dec)# 长宽加倍
    # (32,32)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = UpSampling2D(size=(2, 2))(dec)# 长宽加倍
    # (64,64)
    dec = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = UpSampling2D(size=(2, 2))(dec)# 长宽加倍
    # (128,128)
    dec = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = UpSampling2D(size=(2, 2))(dec)# 长宽加倍
    # (256,256)
    # 最终输出层
    dec = Conv2D(64, (3, 3), strides=(1, 1), input_shape=input_shape, padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(dec)
    dec = BatchNormalization()(dec)
    dec = Conv2D(out_channel, (1, 1), strides=(1, 1), padding='same')(dec)
    # 形状调整层
    #dec = Reshape((1, img_width * img_height))(dec)
    #dec = Permute((2, 1))(dec)# axis=1和axis=2互换位置，等同于np.swapaxes(layer,1,2)
    #dec = Reshape((img_width , img_height, 1))(dec)
    outp_1 = Activation('softmax', name='5_split_Output')(dec)

    #分类分支
    outp_2 = Flatten()(enc)
    outp_2 = Dense(256, activation='relu')(outp_2)
    outp_2 = Dense(5, activation='softmax', name='5_Category_Output')(outp_2)

    model = Model(inp,
                  [outp_1, outp_2])
    return model

In [5]:
# 多阈值平均IoU
"""
单阈值（如0.5）仅反映模型在某一特定定位精度下的表现
而多阈值通过覆盖宽范围的IoU值（如0.5到0.95，步长0.05），综合评估模型在不同定位精度要求下的稳定性
多阈值设计确实能有效避免预测结果处于模糊的中间态
多阈值IoU常用于目标检测（如COCO的IoU@[0.5:0.95]）以评估定位鲁棒性，但语义分割的评估更侧重类别区分而非几何重叠精度
多阈值需存储多个二值化掩膜，如10个阈值将显存占用提升10倍
"""
"""
def dynamic_mean_iou(y_true, y_pred, num_classes=out_channel):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        # 多分类阈值处理
        y_pred_class = tf.cast(y_pred > t, tf.int32)
        y_pred_flat = tf.reshape(y_pred_class, [-1, num_classes])
        y_true_flat = tf.one_hot(tf.reshape(y_true, [-1]), depth=num_classes)
        
        # 动态IoU计算
        intersection = tf.reduce_sum(y_true_flat * y_pred_flat, axis=0)
        union = tf.reduce_sum(y_true_flat, axis=0) + tf.reduce_sum(y_pred_flat, axis=0) - intersection
        iou = (intersection + 1e-7) / (union + 1e-7)  # 1e-7防除零
        
        prec.append(tf.reduce_mean(iou))  # 各阈值mIoU
    
    return K.mean(K.stack(prec), axis=0)
"""    

'\ndef dynamic_mean_iou(y_true, y_pred, num_classes=out_channel):\n    prec = []\n    for t in np.arange(0.5, 1.0, 0.05):\n        # 多分类阈值处理\n        y_pred_class = tf.cast(y_pred > t, tf.int32)\n        y_pred_flat = tf.reshape(y_pred_class, [-1, num_classes])\n        y_true_flat = tf.one_hot(tf.reshape(y_true, [-1]), depth=num_classes)\n        \n        # 动态IoU计算\n        intersection = tf.reduce_sum(y_true_flat * y_pred_flat, axis=0)\n        union = tf.reduce_sum(y_true_flat, axis=0) + tf.reduce_sum(y_pred_flat, axis=0) - intersection\n        iou = (intersection + 1e-7) / (union + 1e-7)  # 1e-7防除零\n        \n        prec.append(tf.reduce_mean(iou))  # 各阈值mIoU\n    \n    return K.mean(K.stack(prec), axis=0)\n'

In [6]:
# 定义损失函数
def dice_coef(y_true, y_pred):
    """Dice系数，用于评估分割性能"""
    smooth = 1.
    y_true_f = K.cast(K.flatten(y_true),dtype = 'float32')
    y_pred_f = K.cast(K.flatten(y_pred),dtype = 'float32')
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def bce_dice_loss(y_true, y_pred):
    """组合损失函数：交叉熵 + Dice损失"""
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)
def bce_dice_loss_2(y_true, y_pred):
    
    return keras.losses.binary_crossentropy(y_true, y_pred)
def bce_dice_loss_3(y_true, y_pred):
    """纯交叉熵损失（用于分类任务）"""
    return keras.losses.categorical_crossentropy(y_true, y_pred)



In [7]:
#设置模型编译参数
optimizer = 'rmsprop'
loss =  {'5_split_Output': bce_dice_loss, # 分割任务损失
         '5_Category_Output': bce_dice_loss_3# 分类任务损失
        }
loss_weights = [5.,1.]# 损失权重（分割:分类 = 5:1）
metrics = {'5_split_Output': keras.metrics.MeanIoU(num_classes=2), # 分割IoU
           '5_Category_Output': 'accuracy'# 分类准确率
          }

# 编译模型
model= keras_model(img_width=img_width, img_height=img_height, out_channel=out_channel)
model.compile(optimizer=optimizer, loss=loss, loss_weights =loss_weights, metrics=metrics)

#加载预训练权重（如果有的话）

if os.path.isfile(os.path.join(aug55Dir, weiPre)):  # os.path.isfile():检查文件是否存在，存在返回true
    try:
        model.load_weights(PretTrWei_path)# 加载预训练权重
        print("预训练权重加载成功")
    except:
        print("预训练权重加载失败")
else:
    print("文件不存在")

# 展示模型结构
model.summary()

文件不存在
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization[0][0]        
_________________________________________________________________________________

In [8]:
"""
定义函数：数据生成器get_train_val_augmented
输入：(validation_split, batch_size, seed)
输出2个生成器：train_generator, val_generator
topDir/
└── working/
    ├── stage1_train/
    │   ├── images/        # 训练图像
    │   └── segmentations/ # 训练分割标签
    └── stage1_val/
        ├── images/        # 测试图像
        └── segmentations/ # 测试分割标签
"""

def get_train_val_augmented(imgDir=imgDir, validation_split=0.3, batch_size=16, seed=seed, out_channel=out_channel):
    """
    数据生成器功能：
    1. 实时数据增强
    2. 分割和分类标签的同步生成
    """
    n_label = 1
    
    # 数据路径配置

    X_train_path = os.path.join(imgDir, "stage1_train", "images")# 训练图像路径
    Y_train_path = os.path.join(imgDir, "stage1_train", "segmentations")# 训练分割标签路径
    X_val_path = os.path.join(imgDir, "stage1_val", "images")# 测试图像路径
    Y_val_path = os.path.join(imgDir, "stage1_val", "segmentations")# 测试分割标签路径
    

    """
    数据增强参数配置
    定义字典data_gen_args和data_gen_args_2，用于配置增强参数。
    """
    data_gen_args = dict(rotation_range=45,# 训练时图像会以中心为轴，在[-45°, 45°]范围内随机旋转，增加模型对物体方向变化的鲁棒性
                         width_shift_range=0.1,# 图像宽度 ±10% 的随机垂直平移
                         height_shift_range=0.1,# 图像高度 ±10% 的随机垂直平移
                         shear_range=0.2,# 沿水平或垂直方向随机施加 ±0.2 弧度 的倾斜变形，模拟视角变化
                         zoom_range=0.2,# 在 [0.8, 1.2] 倍区间内随机缩放图像
                         horizontal_flip=True,# 以 50% 概率水平翻转图像
                         vertical_flip=True,# 以 50% 概率垂直翻转图像
                         fill_mode='reflect') # reflect 表示用图像边缘像素的反射值填充空白，避免出现黑色或纯色边界
    
    data_gen_args_2 = dict(width_shift_range=0.1,# 图像宽度 ±10% 的随机垂直平移
                         height_shift_range=0.1,# 图像高度 ±10% 的随机垂直平移
                         horizontal_flip=True,# 以 50% 概率水平翻转图像
                         vertical_flip=True,# 以 50% 概率垂直翻转图像
                         fill_mode='reflect') # reflect 表示用图像边缘像素的反射值填充空白，避免出现黑色或纯色边界

    
    """
    训练数据生成器
    ImageDataGenerator 是 Keras 库中的一个类，属于深度学习框架中用于图像数据增强的核心工具
    利用字典data_gen_args实例化类ImageDataGenerator，从而获得实例X_datagen用于生成
    
    """
    X_datagen = ImageDataGenerator(**data_gen_args)
    Y_datagen = ImageDataGenerator(**data_gen_args)

    
    #channels = K.image_data_format()
    
    """
    定义数据生成器（函数）train_generator
    输入(seed, X_datagen, Y_datagen, Y_train_path, X_train_path )
    不断的输出张量数组
    当函数中首次遇到 yield 时，函数会 暂停执行，并将 yield 后的值返回给调用者。
    此时，函数的状态（包括局部变量、指令指针等）会被完整保留。
    当生成器被调用 next()再次激活，函数会从上次暂停的位置 恢复执行，直到遇到下一个 yield 或函数结束。
    while True 循环使生成器 无限循环执行，每次循环都会通过 yield 返回一个新的批次数据 X_traini, [Y_traini, C_traini]
    这种设计使得生成器可以持续输出数据，而无需一次性加载全部数据到内存，从而节省内存
    """
    
    def train_generator(seed=seed, X_datagen=X_datagen, Y_datagen=Y_datagen, Y_train_path=Y_train_path, X_train_path = X_train_path):
        """
        训练数据生成流程
        X_datagen 是 ImageDataGenerator 的实例
        flow_from_directory() 是 ImageDataGenerator 类的一个方法，
        用于从指定目录加载图像数据并生成增强后的数据流。调用该方法时，返回的是一个 生成器对象（DirectoryIterator 类的实例）。
        变量X_train_augmented 获得生成器对象的赋值
        """
        # 图像流
        X_train_augmented = X_datagen.flow_from_directory(X_train_path,  # 根目录下需按类别划分子文件夹，每个子文件夹内的图像自动归类为对应类别
                                                          target_size=(img_height, img_width), #将所有图像统一缩放到指定尺寸
                                                          class_mode=None,#不生成标签数据
                                                          color_mode="rgb", #将图像转换为 RGB 三通道格式
                                                          batch_size=batch_size, #定义每个批次的样本数量
                                                          shuffle=True,#打乱数据顺序，防止模型因数据排列产生偏差
                                                          seed=seed#固定随机种子，确保每次运行时数据增强和打乱顺序的一致性（便于复现实验）
                                                         )
        # 标签流
        Y_train_augmented = Y_datagen.flow_from_directory(Y_train_path,  # 根目录下需按类别划分子文件夹，每个子文件夹内的图像自动归类为对应类别
                                                          target_size=(img_height, img_width),
                                                          class_mode="categorical",#生成标签数据，返回 二维的 one-hot 编码标签，适合多分类任务
                                                          color_mode="grayscale", #确保单通道读取
                                                          batch_size=batch_size, 
                                                          shuffle=True,
                                                          seed=seed)
        while True:
            """
            X_datagen 和 Y_datagen 的 flow_from_directory() 使用相同的 seed 和 shuffle 参数，
            确保输入图像和标签的增强操作（如旋转、翻转）完全一致，批次索引严格对应
            """
            X_traini = X_train_augmented.next()
            Y_traini1 = Y_train_augmented.next()
            X_traini = np.array(X_traini, dtype=np.uint8)#将图像数据转换为 uint8（0-255像素值）比 float32 节省 75% 内存
            Y_traini, C_traini = Y_traini1#标签分解：Y_traini1 包含多组标签（分割掩膜 Y_traini 和分类标签 C_traini）
            Y_traini = np.array(Y_traini, dtype=np.bool_)# 标签转换为布尔值（二值掩膜）
            
            
            C_traini_expanded = C_traini[:, np.newaxis, np.newaxis, :]
            C_traini_expanded = np.array(C_traini_expanded, dtype=np.bool_)
            Y_traini = Y_traini * C_traini_expanded
            #result
            Y_traini = np.array(Y_traini, dtype=np.float32)# 如无意外不用这条代码，Y_traini也是float32
            #print(Y_traini.shape, Y_traini.dtype)
            #print(C_traini.shape, C_traini.dtype)
            #max_X = tf.reduce_max(X_traini)
            #min_X = tf.reduce_min(X_traini)
            #print("最大值:", max_X.numpy(), "最小值:", min_X.numpy())

            
            #max_Y = tf.reduce_max(Y_traini)
            #min_Y = tf.reduce_min(Y_traini)
            #print("最大值:", max_Y.numpy(), "最小值:", min_Y.numpy())
            #print(C_traini)
            """
            # 标签维度转换（网页7中的多通道需求）需要原图中以1234标明缺陷标签
            Y_batch = tf.keras.utils.to_categorical(Y_batch, num_classes=out_channel)
            Y_batch = Y_batch.reshape(-1, img_height, img_width, out_channel)
            print("训练集标签范围:", np.unique(Y_traini))
            

            """
            yield X_traini, [Y_traini, C_traini]#返回 [Y_traini, C_traini]，可同时处理多任务学习

    # 验证数据生成器（不进行数据增强）
    X_datagen_val = ImageDataGenerator()
    Y_datagen_val = ImageDataGenerator()

    def val_generator(seed=seed, X_datagen_val=X_datagen_val, Y_datagen_val=Y_datagen_val, Y_val_path=Y_val_path, X_val_path = X_val_path):
        """验证数据生成流程"""
        # 图像流
        X_val_augmented = X_datagen_val.flow_from_directory(X_val_path, 
                                                             target_size=(img_height, img_width), class_mode=None,
                                                             color_mode="rgb", batch_size=batch_size, shuffle=True, seed=seed)
        # 标签流
        Y_val_augmented = Y_datagen_val.flow_from_directory(Y_val_path, 
                                                             target_size=(img_height, img_width),class_mode="categorical",#生成标签数据
                                                             color_mode="grayscale", batch_size=batch_size, shuffle=True,
                                                             seed=seed)
        while True:
            X_vali = X_val_augmented.next()
            Y_vali1 = Y_val_augmented.next()
            X_vali = np.array(X_vali, dtype=np.uint8)
            Y_vali, C_vali = Y_vali1
            Y_vali = np.array(Y_vali, dtype=np.bool_)
            
            C_vali_expanded = C_vali[:, np.newaxis, np.newaxis, :]
            C_vali_expanded = np.array(C_vali_expanded, dtype=np.bool_)
            Y_vali = Y_vali * C_vali_expanded
            #result
            Y_vali = np.array(Y_vali, dtype=np.float32)# 如无意外不用这条代码，Y_vali也是float32
            yield X_vali, [Y_vali, C_vali]

    
    # combine generators into one which yields image and masks
    train_generator = train_generator(seed=seed, X_datagen=X_datagen, Y_datagen=Y_datagen, Y_train_path=Y_train_path, X_train_path = X_train_path)
    val_generator = val_generator(seed=seed, X_datagen_val=X_datagen_val, Y_datagen_val=Y_datagen_val, Y_val_path=Y_val_path, X_val_path = X_val_path)

    return train_generator, val_generator #输出2个生成器



In [9]:
"""
定义函数translate_metric
其作用是将机器学习或数据分析中常用的缩写指标名称（如 'acc' 或 'loss'）转换为全称，用以作为标题输出
旧版Keras（如 <2.3.0）与新版本TensorFlow（≥2.4.0）的API差异可能导致指标名称注册失败：
旧版使用 acc 作为准确率名称，新版统一为 accuracy
"""
def translate_metric(x):
    translations = {'acc': "Accuracy", 
                    'accuracy': "Accuracy", 
                    'loss': "Log-loss (cost function)"}
    if x in translations:
        return translations[x]
    else:
        return x


#import matplotlib.pyplot as plt
#from tensorflow.keras.callbacks import Callback
#from IPython.display import clear_output
"""
自定义一个的Keras回调函数PlotLosses，用于在模型训练过程中实时动态绘制训练集和验证集的指标变化曲线
在每轮（epoch）训练结束后，自动绘制训练集和验证集的损失（loss）或准确率（accuracy）等指标的动态变化曲线，
通过图表直观展示模型收敛情况
"""


class PlotLosses(Callback):#创建新类PlotLosses继承自基类Callback
    """
    在子类中重新定义了初始化方法，这个方法要求在创建实例时需要哪些输入：figsize
    """
    def __init__(self, figsize=None):
        #super(PlotLosses, self).__init__()#兼容 Python 2
        super().__init__()#Python 3+ 特有
        """
        1.super().__init__() 的作用是 调用父类的 __init__ 方法以初始化继承自父类的属性

        2.每个父类的__init__()方法旨在初始化 该类的特有属性 。例如：
        Character类初始化角色名、生命值等；
        People类初始化姓名、年龄、性别等通用属性
        3.当子类 未重写 __init__() 方法时，父类的 __init__() 会被自动继承。
        此时子类实例化时，会直接执行父类的初始化逻辑
        4.如果子类 重写了 __init__() 方法，则父类的 __init__() 会被覆盖，
        子类的初始化逻辑完全由自身定义。此时父类的 __init__() 不会自动执行，除非通过 super() 显式调用
        这里是重新定义了__init__()方法，所以要用该代码显式调用
        5.继承是静态的：通过类定义实现，父类方法默认存在于子类中
        6.调用是动态的：通过 super() 决定是否触发父类的初始化逻辑
        7.子类重写 __init__ 后：父类的 __init__ 仍存在，但需显式调用才能执行
        8.__init__() 是唯一在实例化时由 Python 自动调用的方法，
        这是一个软约束，正是这个功能使得__init__()被称为初始化
        即使未定义，Python 也会隐式调用父类的 __init__()（若存在）
        9.Python 要求 __init__() 必须返回 None，不能显式返回其他值。这是其与普通方法的重要区别
        这是一个硬约束
        """
        
        """
        参数figsize：控制图表大小，例如(16,4)表示宽16英寸、高4英寸
        """
        self.figsize = figsize#给类的实例定义了一个属性：figsize

    """
    定义了类的1个方法
    对于类的方法，最常用的触发方式就是通过实例直接调用：实例.方法
    在 Keras 训练流程中，keras会在某些节点触发回调函数的on_train_begin 和 on_epoch_end 方法
    
    """
    def on_train_begin(self, logs={}):
        """
        定义2个属性base_metrics，logs
        核心意义

        在 Keras 中，如果在模型编译时设置了 metrics 参数（如 accuracy），则：
        训练阶段的指标名称：直接使用定义时的名称（如 loss, accuracy）。
        验证阶段的指标名称：自动添加 val_ 前缀（如 val_loss, val_accuracy）。

        通过过滤掉以 val_ 开头的指标，
        仅保留训练阶段的评估指标，方便后续对训练结果的分析（如绘制训练曲线、记录性能日志等）
        """
        """
        在on_train_begin时self.model.metrics_names经常是空的
        所以，不如在on_epoch_end再建立self.base_metrics
        """
        
        print("self.model.metrics_names:", self.model.metrics_names)
        """
        1.
        self.model 是父类 Callback 提供的属性，指向当前训练或评估的模型实例
        当回调函数被绑定到模型时，Keras框架会通过 set_model() 方法将模型实例赋给 self.model
        这意味着 self.model 是 Callback 类为子类提供的一个接口，用于访问当前操作的模型
        2.
        metrics_names 是 模型实例的属性
        当调用 model.compile(metrics=[...]) 时，
        Keras 会根据 loss 和 metrics 参数生成 metrics_names 列表
        验证集指标会以 val_ 前缀命名（如 val_loss、val_accuracy）
        """
        """
        self.base_metrics = [metric #筛选后的指标名称存入 self.base_metrics 列表
                             for metric #2个metric是一回事，作为一个工具，指代self.model.metrics_names中符合删选条件的元素
                             in self.model.metrics_names #继承自父类：模型的所有评估指标名称列表
                             if not metric.startswith('val_')#排除以 val_ 开头的指标
                            ]
        #self.base_metrics = [metric for metric in self.params['metrics'] if not metric.startswith('val_')]
        
        print("self.base_metrics:", self.base_metrics)
        """
        self.logs = []#logs要在on_train_begin触发时初始化用以存放之后产生的训练log

    def on_epoch_end(self, epoch, logs={}):
        self.base_metrics = [metric #筛选后的指标名称存入 self.base_metrics 列表
                             for metric #2个metric是一回事，作为一个工具，指代self.model.metrics_names中符合删选条件的元素
                             in self.model.metrics_names #继承自父类：模型的所有评估指标名称列表
                             if not metric.startswith('val_')#排除以 val_ 开头的指标
                            ]
        #self.base_metrics = [metric for metric in self.params['metrics'] if not metric.startswith('val_')]
        print("self.base_metrics:", self.base_metrics)
        self.logs.append(logs.copy())# 记录当前epoch的指标值
        """
        使用clear_output(wait=True)清除旧图表并重新绘制，实现在同一位置刷新图像（适用于Jupyter Notebook等交互环境）
        """
        clear_output(wait=True)
        plt.figure(figsize=self.figsize)# 初始化画布

        for metric_id, metric in enumerate(self.base_metrics):
            plt.subplot(1, len(self.base_metrics), metric_id + 1)

            plt.plot(range(1, len(self.logs) + 1),
                     [log[metric] for log in self.logs],
                     label="training")
            #if self.params['do_validation']:
            """
            self.params['do_validation'] 是一个布尔值参数，用于指示模型在训练时是否执行验证步骤。其值由以下条件决定：
            若在 model.fit() 中显式传入 validation_data 或设置 validation_split，则值为 True；
            若未提供任何验证数据，则值为 False。
            """
            has_validation = any(key.startswith('val_') for key in logs.keys())
            if has_validation:
                plt.plot(range(1, len(self.logs) + 1),
                         [log['val_' + metric] for log in self.logs],
                         label="validation")
            #plt.title(translate_metric(metric))
            #plt.title(metric.capitalize())  # 标题优化（替代translate_metric）
            plt.xlabel('epoch')#
            plt.legend(loc='center left')

        plt.tight_layout()
        plt.show();#展示图表
        #plt.pause(0.1)  # 允许图像更新

plot_losses = PlotLosses(figsize=(16, 4))

In [10]:
# 训练配置
callbacks_list = [
    plot_losses,# 可视化回调
    # 模型动态保存
    keras.callbacks.ModelCheckpoint(
        filepath = os.path.join(aug55Dir, weiDur),
        monitor='val_loss',
        save_best_only=True,
    ),
    # 动态学习率调整
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1, patience=10
    )
]

In [11]:
# 启动训练
batch_size = 2
# 批大小：每次迭代（iteration）输入到模型中的样本数量。每个batch后更新模型参数
#小批量（如32）引入随机性，可能帮助模型跳出局部最小值；大批量（如1024）提高计算效率但可能降低泛化能力
#显存不足时需减小batch_size
epochs=100
# 轮次：整个训练数据集被完整遍历一次的次数。每个epoch结束后评估模型表现
# 过少（如10）可能导致欠拟合，过多（如1000）易导致过拟合
callbacks = callbacks_list
validation_split=0.3

train_generator, val_generator = get_train_val_augmented(imgDir=imgDir, validation_split=validation_split, batch_size=batch_size, seed=seed)

model.fit(x=train_generator,
                    validation_data=val_generator,
                    steps_per_epoch=1,#len(X_train1)/(batch_size*2),
                    validation_steps=1,#len(C_val1)/epochs,
                    epochs=epochs,verbose=0,
          callbacks=[plot_losses])
          #callbacks=callbacks_list)

# 保存最终模型为hdf5文件
model_out = model
model_out.save_weights(filepath=os.path.join(aug55Dir, weiEnd))

KeyboardInterrupt: 